In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
!git clone https://github.com/jannisborn/covid19_ultrasound.git

Cloning into 'covid19_ultrasound'...
remote: Enumerating objects: 4503, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 4503 (delta 21), reused 20 (delta 6), pack-reused 4453
Receiving objects: 100% (4503/4503), 165.47 MiB | 37.53 MiB/s, done.
Resolving deltas: 100% (353/353), done.
Updating files: 100% (3872/3872), done.


In [3]:
meta_data = pd.read_csv('./covid19_ultrasound/data/dataset_metadata.csv')

In [4]:
meta_data[meta_data['InThisRepo'] == 'Yes']

,Date added,Current location,Filename,URL (Video Name),Label,Lung Severity Score,License,LinkToLicense,ChangesMade,InThisRepo,...,Healthy,Fever,Cough,Respiratory problems,Headache,Loss of smell/taste,Fatigue,Sore throat,Asymptomatic,Other
21,20.04.20,data/pocus_videos/Convex,Cov-Atlas+(44),http://www.thepocusatlas.com/covid19-1/m2ra6cg...,COVID-19,2.0,CC BY-NC 4.0,https://creativecommons.org/licenses/by-nc/4.0/#,Cropping,Yes,...,0.0,1,0,1,0,0,0,0,0,NaN
22,20.04.20,data/pocus_videos/Convex,Cov-Atlas+(45),http://www.thepocusatlas.com/covid19-1/3ytz1el...,COVID-19,3.0,CC BY-NC 4.0,https://creativecommons.org/licenses/by-nc/4.0/#,Cropping,Yes,...,0.0,1,0,1,0,0,0,0,0,NaN
23,20.04.20,data/pocus_videos/Convex,Cov-Atlas-Day+2,http://www.thepocusatlas.com/covid19 (Day2),COVID-19,1.0,CC BY-NC 4.0,https://creativecommons.org/licenses/by-nc/4.0/#,Cropping,Yes,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
24,20.04.20,data/pocus_videos/Convex,Cov-Atlas-Day+4,http://www.thepocusatlas.com/covid19 (Day4),COVID-19,3.0,CC BY-NC 4.0,https://creativecommons.org/licenses/by-nc/4.0/#,Cropping,Yes,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
25,20.04.20,data/pocus_videos/Convex,Pneu-Atlas-pneumonia,http://www.thepocusatlas.com/ (Classic Finding...,Bacterial pneumonia,NaN,CC BY-NC 4.0,https://creativecommons.org/licenses/by-nc/4.0/#,Cropping,Yes,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,19.01.21,pocus_images/convex,Cov_Oliviera_2020_Fig15A,https://www.scielo.br/img/revistas/rb/v53n4//0...,COVID-19,NaN,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
365,19.01.21,pocus_images/linear,Cov_Oliviera_2020_Fig16,https://www.scielo.br/img/revistas/rb/v53n4//0...,COVID-19,NaN,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
371,19.01.21,pocus_videos/convex,Cov_emdocs_vid1,http://www.emdocs.net/wp-content/uploads/2020/...,COVID-19,NaN,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
372,19.01.21,pocus_videos/linear,Cov_emdocs_vid2,http://www.emdocs.net/wp-content/uploads/2020/...,COVID-19,3.0,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN


In [5]:
base_dir = "./covid19_ultrasound/data/"
out_table = pd.DataFrame(index=['cov', 'pne', 'reg', 'vir'])
for modality in ["convex", "linear"]:
    for datatype in ["videos", "images"]:
        path = os.path.join(base_dir, "pocus_" + datatype, modality)
        file_list = filter(
            lambda x: (
                x[0] != "." and os.path.isfile(os.path.join(path, x)) and
                not "_Butterfly" in x
            ), os.listdir(path)
        )
        label_list = [f[:3].lower() for f in file_list]
        counts = [
            label_list.count(lab) for lab in ['cov', 'pne', 'reg', 'vir']
        ]
        out_table[modality + "_" + datatype] = counts
print("Data without Butterfly:")
print(out_table)

Data without Butterfly:
     convex_videos  convex_images  linear_videos  linear_images
cov             30              5              3              3
pne             36             14              1              0
reg             56             10              3              0
vir              2              0              3              0


In [6]:
meta_data

,Date added,Current location,Filename,URL (Video Name),Label,Lung Severity Score,License,LinkToLicense,ChangesMade,InThisRepo,...,Healthy,Fever,Cough,Respiratory problems,Headache,Loss of smell/taste,Fatigue,Sore throat,Asymptomatic,Other
0,20.04.20,butterfly,Cov-Butterfly-COVID Lung 2,https://www.butterflynetwork.com/covid-19#lung...,COVID-19,NaN,None,N.A.,"Cropping, Artifact removal, frame subselection",No,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
1,20.04.20,butterfly,Cov-Butterfly-Skip Lesion,https://www.butterflynetwork.com/covid19/covid...,COVID-19,NaN,None,N.A.,"Cropping, Artifact removal, frame subselection",No,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
2,20.04.20,butterfly,Cov-Butterfly-Confluent B lines,https://www.butterflynetwork.com/covid19/covid...,COVID-19,2.0,None,N.A.,"Cropping, Artifact removal, frame subselection",No,...,0.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
3,20.04.20,butterfly,Reg-Butterfly-Normal Lung A lines,https://www.butterflynetwork.com/covid19/covid...,regular,NaN,None,N.A.,"Cropping, Artifact removal, frame subselection",No,...,1.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
4,20.04.20,butterfly,Reg-Butterfly-Normal Lung_Example 2,https://www.butterflynetwork.com/covid19/covid...,regular,0.0,None,N.A.,"Cropping, Artifact removal, frame subselection",No,...,1.0,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,19.01.21,pocus_videos/convex,Cov_wfumb_case_dez,https://www.youtube.com/watch?v=CpXPimphNSM&fe...,COVID-19,NaN,None,N.A.,Cropping,No,...,NaN,0,0,0,0,0,0,0,1,NaN
370,19.01.21,pocus_videos/convex,Cov_Arnthfield_2020_Vid3,https://www.medrxiv.org/content/10.1101/2020.1...,COVID-19,NaN,CC-BY-NC-ND,https://creativecommons.org/licenses/by-nc-nd/...,Cropping,No,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
371,19.01.21,pocus_videos/convex,Cov_emdocs_vid1,http://www.emdocs.net/wp-content/uploads/2020/...,COVID-19,NaN,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN
372,19.01.21,pocus_videos/linear,Cov_emdocs_vid2,http://www.emdocs.net/wp-content/uploads/2020/...,COVID-19,3.0,CC BY 4.0,https://creativecommons.org/licenses/by/4.0/,Cropping,Yes,...,NaN,n/A,n/A,n/A,n/A,n/A,n/A,n/A,n/A,NaN


In [7]:
def label_enc(idx):
    if idx == 'COVID-19':
        return 2
    elif idx == 'Bacterial pneumonia':
        return 1
    elif idx == 'Viral pneumonia':
        return 1
    else:
        return 0
meta_data['Label'] = meta_data['Label'].apply(label_enc)

In [8]:
meta_data['InThisRepo']

0       No
1       No
2       No
3       No
4       No
      ... 
369     No
370     No
371    Yes
372    Yes
373    Yes
Name: InThisRepo, Length: 374, dtype: object

In [9]:
possible_endings = [
       ".mp4", ".mpeg", ".avi", ".gif",
    ".mov"
]
base = './covid19_ultrasound/data/'
table = meta_data
#table = table[table['InThisRepo'] == 'Yes']
real_paths = []
labels = []
in_this_repos = []
for i in range(len(table)):
    fn = table.iloc[i]["Filename"]
    location = table.iloc[i]["Current location"].lower()
    if pd.isnull(location) or 'data_with_artifacts' in location:
        continue
    if location.startswith("data/"):
        location = location[5:]

    path1 = os.path.join(base, location, fn)
    path2 = ".".join(path1.split(".")[:-1]) + ".mp4"
    for ending in possible_endings:
        if os.path.exists(path1 + ending):
            table = table.copy()
            real_path = path1+ending
            label = table['Label'].iloc[i]
            in_this_repo = table['InThisRepo'].iloc[i]
            
            real_paths.append(real_path)
            labels.append(label)
            in_this_repos.append(in_this_repo)

preprocessed_metadata = pd.DataFrame({'path':real_paths,'label':labels,'in_this_repo':in_this_repos})

In [10]:
vid_path = preprocessed_metadata['path'].iloc[0]

In [11]:
vidcap = cv2.VideoCapture(vid_path)
success,image = vidcap.read()

In [12]:
preprocessed_metadata

,path,label,in_this_repo
0,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
1,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
2,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
3,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
4,./covid19_ultrasound/data/pocus_videos/convex/...,1,Yes
...,...,...,...
127,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
128,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
129,./covid19_ultrasound/data/pocus_videos/convex/...,2,Yes
130,./covid19_ultrasound/data/pocus_videos/linear/...,2,Yes


In [13]:
!mkdir data

In [14]:
paths = []
labels = []
video_ids = []
for i in tqdm(range(len(preprocessed_metadata))):
    vidcap = cv2.VideoCapture(preprocessed_metadata['path'].iloc[i])
    success,image = vidcap.read()
    count = 0
    while success:
        path = f"./data/Video_{i}_frame_%d.jpg" % count
        video_id = f'Video_{i}'
        cv2.imwrite(path, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        print('Read a new frame: ', success)
        count += 1
        paths.append(path)
        video_ids.append(video_id)
        labels.append(preprocessed_metadata['label'].iloc[i])
        
metadata_final = pd.DataFrame({'path':paths,'label':labels,'video_id':video_ids})

  0%|          | 0/132 [00:00<?, ?it/s]

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True


  2%|▏         | 3/132 [00:00<00:07, 17.27it/s]

Read a new frame:  True
Read a new frame:  False
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new fra

  4%|▍         | 5/132 [00:00<00:09, 12.78it/s]

...

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

 99%|█████████▉| 131/132 [01:44<00:01,  1.47s/it]

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new fram

100%|██████████| 132/132 [01:45<00:00,  1.25it/s]

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False


In [15]:
metadata_final

,path,label,video_id
0,./data/Video_0_frame_0.jpg,2,Video_0
1,./data/Video_0_frame_1.jpg,2,Video_0
2,./data/Video_0_frame_2.jpg,2,Video_0
3,./data/Video_0_frame_3.jpg,2,Video_0
4,./data/Video_0_frame_4.jpg,2,Video_0
...,...,...,...
19969,./data/Video_131_frame_70.jpg,2,Video_131
19970,./data/Video_131_frame_71.jpg,2,Video_131
19971,./data/Video_131_frame_72.jpg,2,Video_131
19972,./data/Video_131_frame_73.jpg,2,Video_131


In [16]:
metadata_final.to_csv('covid_data.csv',index=False)